In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
    FunctionTransformer,
)

# Raw - Mobility & Air quality

Importación de datos de calidad de aire y mobilidad

inicialmente se leen los datos crudos desde los archivos descargables de las  respectivas fuentes, se procede a compilar todos los datos de calidad de aire en 3 dataframe diferentes (Train, Test y Validation) que se convierten posteriormente a un archivo HDF5 el cual permite recolectar la información de manera más eficiente facilitando la lectura para proximas ocasiones, de igual maneraa se procede con los datos de mobilidad 

In [2]:
''' 
directory_path = 'C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Air Quality/Train'
dfs = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

Train_air = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to an HDF5 file
Train_air.to_hdf('C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Air Quality/Train/train_air_dataframe.h5', key='df', mode='w')
''' 

" \ndirectory_path = 'C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Air Quality/Train'\ndfs = []\n\nfor filename in os.listdir(directory_path):\n    if filename.endswith('.csv'):\n        file_path = os.path.join(directory_path, filename)\n        df = pd.read_csv(file_path)\n        dfs.append(df)\n\nTrain_air = pd.concat(dfs, ignore_index=True)\n\n# Save the combined DataFrame to an HDF5 file\nTrain_air.to_hdf('C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Air Quality/Train/train_air_dataframe.h5', key='df', mode='w')\n"

In [3]:
# Load the DataFrame from the HDF5 file
Train_air = pd.read_hdf('C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Air Quality/Train/train_air_dataframe.h5', key='df')

In [4]:
Train_air.head()

,Fecha_Hora,codigoSerial,pm25,calidad_pm25,pm10,calidad_pm10,pm1,calidad_pm1,no,calidad_no,...,p_ssr,calidad_p_ssr,pliquida_ssr,calidad_pliquida_ssr,rglobal_ssr,calidad_rglobal_ssr,taire10_ssr,calidad_taire10_ssr,vviento_ssr,calidad_vviento_ssr
0,2021-01-01 00:00:00,12,31.0,1.0,49.0,1.0,-9999.0,1.0,-9999.0,4.31,...,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0
1,2021-01-01 01:00:00,12,41.0,1.0,51.0,1.0,-9999.0,1.0,-9999.0,4.31,...,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0
2,2021-01-01 02:00:00,12,38.0,1.0,51.0,1.0,-9999.0,1.0,-9999.0,4.31,...,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0
3,2021-01-01 03:00:00,12,30.0,1.0,42.0,1.0,-9999.0,1.0,-9999.0,4.31,...,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0
4,2021-01-01 04:00:00,12,29.0,1.0,39.0,1.0,-9999.0,1.0,-9999.0,4.31,...,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0,-9999.0,1.0


In [5]:
Train_air.columns

Index(['Fecha_Hora', 'codigoSerial', 'pm25', 'calidad_pm25', 'pm10',
       'calidad_pm10', 'pm1', 'calidad_pm1', 'no', 'calidad_no', 'no2',
       'calidad_no2', 'nox', 'calidad_nox', 'ozono', 'calidad_ozono', 'co',
       'calidad_co', 'so2', 'calidad_so2', 'pst', 'calidad_pst', 'dviento_ssr',
       'calidad_dviento_ssr', 'haire10_ssr', 'calidad_haire10_ssr', 'p_ssr',
       'calidad_p_ssr', 'pliquida_ssr', 'calidad_pliquida_ssr', 'rglobal_ssr',
       'calidad_rglobal_ssr', 'taire10_ssr', 'calidad_taire10_ssr',
       'vviento_ssr', 'calidad_vviento_ssr'],
      dtype='object')

In [6]:
Air = Train_air.copy()
Air = Train_air[['Fecha_Hora', 'codigoSerial', 'pm25', 'calidad_pm25']]

Del dataset anterior unicamente se tomaran en cuanta las columnas de fecha_hora, codigoSerial, pm25 y calidad_pm25, ya que, las demás columnas al tener valores de -9999 y calidad 1 significan que los sensores están fuera de servicio y por la descripción del dataset se evidencia que solo las columnas de pm25 tienen un comportamiento diferente al mencionado

In [7]:
Air.loc[:,'Fecha_Hora'] = pd.to_datetime(Air['Fecha_Hora'], format='%Y-%m-%d %H:%M:%S')

In [8]:
# Establecer la columna 'fecha' como índice
Air.set_index('Fecha_Hora', inplace=True)

c:\Users\MARIA PAULA\.conda\envs\base3\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [9]:
Air.head(10)

,codigoSerial,pm25,calidad_pm25
Fecha_Hora,,,
2021-01-01 00:00:00,12,31.0,1.0
2021-01-01 01:00:00,12,41.0,1.0
2021-01-01 02:00:00,12,38.0,1.0
2021-01-01 03:00:00,12,30.0,1.0
2021-01-01 04:00:00,12,29.0,1.0
2021-01-01 05:00:00,12,33.0,1.0
2021-01-01 06:00:00,12,26.0,1.0
2021-01-01 07:00:00,12,30.0,1.0
2021-01-01 08:00:00,12,31.0,1.0


La columna codigoSerial representa el codigo de la estacion con esta podemos saber donde se encuentra ubicada, para poder comparar los datos con el dataset de mobilidad

Comuna 10
CEN-TRAF - Centro - Plaza Botero


Comuna 16
Belén Altavista - I.E Pedro Octavio Amado
Belén - I.E Pedro Justo Barrio


Comuna 8
Villahermosa - Planta de Producción Agua Potable EPM


Comuna 14
El Poblado - I.E INEM Sede Santa Catalina


Comuna 13
San Cristóbal - Parque Biblioteca Fernando Botero


Comuna 4
Aranjuez - I.E Ciro Mendia


https://www.medellin.gov.co/es/conoce-algunos-datos-generales-de-la-ciudad/


In [10]:
Air['codigoSerial'].value_counts()

codigoSerial
12    21144
79    21144
80    21144
83    21144
84    21144
85    21144
86    20400
Name: count, dtype: int64

In [11]:
def missing_values(df):
    Column_Names = df.columns

    Total_rows = df.shape[0]

    missing_values = df.isnull().sum()

    Percent_missing = (missing_values / Total_rows)*100
    
    result = pd.DataFrame({
        'Column_Name': Column_Names,
        'Missing_Values': missing_values.values,
        'Total_Rows': Total_rows,
        'Percent_Missing': Percent_missing.values
    })

    return result

In [12]:
Air_organized = Air.pivot_table(index=Air.index, columns='codigoSerial', values=['pm25', 'calidad_pm25'], aggfunc = 'mean')

In [13]:
Air_organized.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21144 entries, 2021-01-01 00:00:00 to 2023-05-31 23:00:00
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   (calidad_pm25, 12)  21144 non-null  float64
 1   (calidad_pm25, 79)  21144 non-null  float64
 2   (calidad_pm25, 80)  21144 non-null  float64
 3   (calidad_pm25, 83)  21144 non-null  float64
 4   (calidad_pm25, 84)  21144 non-null  float64
 5   (calidad_pm25, 85)  21144 non-null  float64
 6   (calidad_pm25, 86)  20400 non-null  float64
 7   (pm25, 12)          21144 non-null  float64
 8   (pm25, 79)          21144 non-null  float64
 9   (pm25, 80)          21144 non-null  float64
 10  (pm25, 83)          21144 non-null  float64
 11  (pm25, 84)          21144 non-null  float64
 12  (pm25, 85)          21144 non-null  float64
 13  (pm25, 86)          20400 non-null  float64
dtypes: float64(14)
memory usage: 2.4 MB


In [14]:
missing_values(Air_organized)

,Column_Name,Missing_Values,Total_Rows,Percent_Missing
0,"(calidad_pm25, 12)",0,21144,0.000000
1,"(calidad_pm25, 79)",0,21144,0.000000
2,"(calidad_pm25, 80)",0,21144,0.000000
3,"(calidad_pm25, 83)",0,21144,0.000000
4,"(calidad_pm25, 84)",0,21144,0.000000
5,"(calidad_pm25, 85)",0,21144,0.000000
6,"(calidad_pm25, 86)",744,21144,3.518729
7,"(pm25, 12)",0,21144,0.000000
8,"(pm25, 79)",0,21144,0.000000
9,"(pm25, 80)",0,21144,0.000000


Para los datos nulos que se encuentran en los datos asociados a la estación 86, se tienen 2 alternativas

1. **Forward or backward fill:** Use the value from the previous or next observation to fill null values. This method is especially useful for time series data.

df.fillna(method='ffill', inplace=True, axis=0)

2. **Interpolation:** Estimate missing values based on other values in the dataset. Pandas provides various interpolation methods such as linear, polynomial, or spline interpolation.

df.interpolate(method='linear', inplace=True, axis=0)

Inicialmente se llevara a cabo **Forward or backward fill** pues es más acorde al tipo de datos

In [15]:
"""
# Forward fill null values
#Air_organized.fillna(method='ffill', inplace=True, axis=0)
Air_organized.ffill(axis=0, inplace=True)
"""

"\n# Forward fill null values\n#Air_organized.fillna(method='ffill', inplace=True, axis=0)\nAir_organized.ffill(axis=0, inplace=True)\n"

In [16]:
#missing_values(Air_organized)

In [17]:
# Seleccionar todas las columnas con el nombre 'calidad_pm25' en el primer nivel del MultiIndex
calidad_pm25_columns = Air_organized.xs('calidad_pm25', level=0, axis=1)

# Iterar sobre cada columna (por código serial)
for column in calidad_pm25_columns.columns:
    # Obtener los valores únicos en cada columna
    unique_vals = calidad_pm25_columns[column].unique()
    
    # Contar la frecuencia de cada valor único
    value_counts = calidad_pm25_columns[column].value_counts()
    
    # Mostrar los resultados
    print(f"Columna (código serial): {column}")
    print(value_counts)
    print("\n")


Columna (código serial): 12
12
1.0000    19267
4.3100      473
2.6000      373
2.1000      322
2.3000      233
3.1000      233
4.2985      115
2.2000       88
3.0000       32
2.4000        4
2.0000        3
4.2995        1
Name: count, dtype: int64


Columna (código serial): 79
79
1.00    19952
2.30      252
3.10      238
3.30      186
4.32      120
4.31      115
3.00      109
2.20       99
2.40       70
2.00        1
3.20        1
2.60        1
Name: count, dtype: int64


Columna (código serial): 80
80
1.00    19745
3.00      429
2.30      243
2.40      177
3.10      171
4.31      117
2.20      115
3.30       78
4.32       68
3.20        1
Name: count, dtype: int64


Columna (código serial): 83
83
1.00    20113
2.30      256
4.31      197
3.10      160
4.32      107
2.20      102
3.30       87
2.40       66
3.00       52
3.20        4
Name: count, dtype: int64


Columna (código serial): 84
84
1.00    19967
2.30      204
4.31      179
3.00      175
3.10      172
2.40      144
3.30     

Desde el diccionario de datos del SIATA menciona que valores de calidad mayores a 2.6 no son confiables por lo que se eliminan de mi dataset en aras de mejorar la calidad de mis datos y por ende la predicción

In [18]:
prueba = Air_organized.copy()

# Filtrar todas las columnas de 'calidad_pm25'
calidad_columns = prueba.xs('calidad_pm25', level=0, axis=1)

# Iterar sobre cada código serial en las columnas 'calidad_pm25'
for codigoSerial in calidad_columns:
    # Reemplazar los valores en 'pm25' si 'calidad_pm25' es menor a 2.6
    prueba.loc[prueba[('calidad_pm25', codigoSerial)] > 2.6, ('pm25', codigoSerial)] = np.nan


In [19]:
missing_values(prueba)

,Column_Name,Missing_Values,Total_Rows,Percent_Missing
0,"(calidad_pm25, 12)",0,21144,0.000000
1,"(calidad_pm25, 79)",0,21144,0.000000
2,"(calidad_pm25, 80)",0,21144,0.000000
3,"(calidad_pm25, 83)",0,21144,0.000000
4,"(calidad_pm25, 84)",0,21144,0.000000
5,"(calidad_pm25, 85)",0,21144,0.000000
6,"(calidad_pm25, 86)",744,21144,3.518729
7,"(pm25, 12)",854,21144,4.038971
8,"(pm25, 79)",769,21144,3.636966
9,"(pm25, 80)",864,21144,4.086266


| Rango de PM2.5 (µg/m³) | Nivel de Salud                     |
|--------------------------|------------------------------------|
| 0 - 50                   | Saludable                          |
| 51 - 100                 | Moderado                           |
| 101 - 150                | Poco saludable para grupos sensibles |
| 151 - 200                | Poco saludable para la población en general |
| Más de 200               | Muy poco saludable o peligroso     |


# Trusted - Air Quality

In [2]:
#Air_organized.to_hdf('C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Programacion/air_organized.h5', key='df', mode='w')

# Load the DataFrame from the HDF5 file
Air_organized = pd.read_hdf('air_organized.h5', key='df')

In [ ]:
'''
import matplotlib.pyplot as plt
ncols = 3
# Crear subgráficos para cada columna en el DataFrame
fig, axes = plt.subplots(nrows=5, ncols=ncols, figsize=(15, 15))

# Iterar sobre cada columna y crear un histograma en su respectivo subplot
for i, columna in enumerate(Air_organized.columns):
    fila = i // ncols  # Calcular la fila actual
    columna_subplot = i % ncols  # Calcular la columna actual
    axes[fila, columna_subplot].hist(Air_organized[columna], bins=15, color='skyblue', alpha=0.7)
    axes[fila, columna_subplot].set_title(columna)
    axes[fila, columna_subplot].set_xlabel('Valor')
    axes[fila, columna_subplot].set_ylabel('Frecuencia')

# Ajustar el diseño de los subgráficos
plt.tight_layout()

# Mostrar los subgráficos
plt.show()
'''

In [ ]:
Air_organized.describe()

### Manejo de Outliers
To find outliers in each column of your dataset using a clustering approach, iI use density-based clustering algorithms, in this case DBSCAN (Density-Based Spatial Clustering of Applications with Noise).
where I have 2 hyperparameters:

* eps: Controls the size of the neighborhood around each point.

* min_samples: Determines how many points need to be within the neighborhood for a point to be considered a core point.


Una vez detectados los outlier se cambia su valor por el valor del quantile 90%, esto para denotar que es un valor alto sin embargo no el que se muestra fuera de escala como estaba originalmente, por otro lado, aquellos valores negativos que se encuentran se cambian a cero puesto que no tiene sentido que haya valores negativos

In [3]:
from sklearn.cluster import DBSCAN

Air_median = Air_organized.copy()

# Apply DBSCAN to each column
outlier_indices = {}
for col in Air_organized.columns[1:]:  # Excluding the 'codigoSerial' column
    X = np.array(Air_organized[col]).reshape(-1, 1)  # Reshape to meet input requirements of DBSCAN
    dbscan = DBSCAN(eps=20, min_samples=1000) 
    dbscan.fit(X)
    # Convert indices to datetime values
    outlier_indices[col] = Air_organized.index[dbscan.labels_ == -1]

    # Replace outliers with the 0.9 quantile value of each column
    quantile_value = Air_organized[col].quantile(0.9)
    # Iterate over the outlier indices and replace the values
    for index in outlier_indices[col]:
        # Check if the value is negative, and if so, change it to zero
        if Air_median.at[index, col] < 0:
            Air_median.at[index, col] = 0
        else:
            Air_median.at[index, col] = quantile_value


In [ ]:
'''
# Count outliers for each column
outlier_counts = {}

for col in Air_organized.columns[1:]:  # Exclude the "codigoSerial" column
    outlier_counts[col] = len(outlier_indices[col])

# Print outlier counts for each column
for col, count in outlier_counts.items():
    print(f"Number of outliers in {col}: {count}")
'''

In [ ]:
'''
import matplotlib.pyplot as plt

# Determine the number of columns and rows for the grid
num_cols = 3  # Number of columns in the grid
num_rows = (len(Air_organized.columns) - 1) // num_cols + 1  # Number of rows needed

# Plot each time series along with outliers in a grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

for i, col in enumerate(Air_organized.columns[1:]):  # Exclude the "codigoSerial" column
    row_idx = i // num_cols
    col_idx = i % num_cols
    
    axes[row_idx, col_idx].plot(Air_organized.index, Air_organized[col], label=col)
    
    # Plot outliers
    outliers = Air_organized.loc[outlier_indices[col], col]
    axes[row_idx, col_idx].scatter(outliers.index, outliers, color='red', label='Outliers')
    
    axes[row_idx, col_idx].set_title(f"{col} Time Series with DBSCAN Outliers")
    axes[row_idx, col_idx].set_xlabel("Date")
    axes[row_idx, col_idx].set_ylabel("Value")
    axes[row_idx, col_idx].legend()

# Hide empty subplots if any
for i in range(len(Air_organized.columns) % num_cols, num_cols):
    axes[num_rows - 1, i].axis('off')

plt.tight_layout()
plt.show()
'''

In [4]:
for col in Air_median.columns:
    # Replace negative values with zero
    Air_median[col] = Air_median[col].apply(lambda x: 0 if x < 0 else x)

In [ ]:
Air_median.describe()

In [ ]:
import matplotlib.pyplot as plt
# Define the number of rows and columns for the grid
num_rows = 5
num_cols = (len(Air_median.columns) - 1) // num_rows + 1  # Excluding the 'codigoSerial' column

# Create a grid of plots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each column
for i, col in enumerate(Air_median.columns[1:]):  # Exclude the 'codigoSerial' column
    ax = axes[i] if i < len(axes) else None
    ax.plot(Air_median.index, Air_median[col], marker='o', linestyle='-')
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel(col)
    ax.grid(True)

# Hide any empty subplots
for ax in axes[len(Air_median.columns[1:]):]:
    if ax is not None:
        ax.axis('off')

# Adjust layout and display the plots
plt.tight_layout()
plt.show()

In [ ]:
Air_median.describe()

Guardo los datos transformados

In [ ]:
# Specify the file path
file_path = 'C:/Users/MARIA PAULA/Downloads/EAFIT/Proyecto de Grado/Programacion/Air_median.h5'

# Save DataFrame to HDF5 file
Air_median.to_hdf(file_path, key='df', mode='w')

Grafico la nueva distribución de los datos luego de la limpieza

In [ ]:
ncols = 3
# Crear subgráficos para cada columna en el DataFrame
fig, axes = plt.subplots(nrows=5, ncols=ncols, figsize=(15, 15))

# Iterar sobre cada columna y crear un histograma en su respectivo subplot
for i, columna in enumerate(Air_median.columns):
    fila = i // ncols  # Calcular la fila actual
    columna_subplot = i % ncols  # Calcular la columna actual
    axes[fila, columna_subplot].hist(Air_median[columna], bins=15, color='skyblue', alpha=0.7)
    axes[fila, columna_subplot].set_title(columna)
    axes[fila, columna_subplot].set_xlabel('Valor')
    axes[fila, columna_subplot].set_ylabel('Frecuencia')

# Ajustar el diseño de los subgráficos
plt.tight_layout()

# Mostrar los subgráficos
plt.show()

# Trusted -  Mobility 

In [18]:
# Load the DataFrame from the HDF5 file
Mobilidad = pd.read_hdf('combined_dataframe.h5', key='df')

In [ ]:
Mobilidad.info()

In [ ]:
Mobilidad.set_index('Fecha Trafico')

In [ ]:
Mobilidad['codigo comuna'].unique()

In [ ]:
Mob = Mobilidad.copy()
Mob.columns


In [ ]:
Mob = Mob[['Fecha Trafico', 'Velocidad (Km/h)', 'Operación',
       'Intensidad', 'Nº vehiculos long 1', 'Nº vehiculos long 2',
       'Nº vehiculos long 3', 'Ocupación', 'codigo comuna',
       'nombre comuna']]

In [ ]:
Mob.set_index('Fecha Trafico')

VOY ACA, falta quitar las comunas que no necesito, arreglar nulos como lo hice en Air_organized y mirar como unir ambos dataframes para sacar la matriz de correlacion, las vbles categoricas de Mob hay que codificarlas tambien

In [ ]:
Features = pd.DataFrame(Mobilidad.drop(['Wage', 'Name'], axis=1))

numerical_fea = Features.select_dtypes(include = np.number).columns
print('Caracteristicas numericas: ', numerical_fea)

categorical_fea = Features.select_dtypes(include = 'object').columns
print('\n','Caracteristicas numericas: ', categorical_fea)

In [23]:
Mob = Mobilidad[['Velocidad (Km/h)',
       'Intensidad', 'Nº vehiculos long 1', 'Nº vehiculos long 2',
       'Nº vehiculos long 3', 'Ocupación', 'Longitud',
       'Latitud', 'codigo comuna']]

In [ ]:
Mob.info()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = Mob.corr()

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Create a heatmap with Seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# Add a title
plt.title('Correlation Matrix')

# Rotate the tick labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()